In [1]:
import time
from flask import Flask, request, jsonify
from collections import namedtuple
import pandas as pd
import json
from pytrends.request import TrendReq
import requests
from statistics import mean
import sys
from tqdm import tqdm

### nxn+1 distance+popularity matrix


In [2]:
pytrend = TrendReq()
startTime = time.time()

In [3]:
cities = pd.read_csv('cities_20k.csv')

In [4]:
cities.head()

,Unnamed: 0,Woonplaatsen,Provincie,Landsdeel,Gemeente,Latitude,Longitude,Population,Coordinates
0,12,Assen,Drenthe,Noord-Nederland,Assen,52.983333,6.550000,68798,"52° 59′ NB, 6° 33′ OL"
1,29,Coevorden,Drenthe,Noord-Nederland,Coevorden,52.650000,6.733333,35303,"52° 39′ NB, 6° 44′ OL"
2,64,Emmen,Drenthe,Noord-Nederland,Emmen,52.783333,6.900000,56545,"52° 47′ NB, 6° 54′ OL"
3,98,Hoogeveen,Drenthe,Noord-Nederland,Hoogeveen,52.716667,6.466667,40190,"52° 43′ NB, 6° 28′ OL"
4,115,Meppel,Drenthe,Noord-Nederland,Meppel,52.700000,6.183333,34125,"52° 42′ NB, 6° 11′ OL"


In [5]:
city_names = cities['Woonplaatsen'].unique()
len_city_names = len(city_names)
popularity = [0] * len_city_names

In [6]:
new_df = pd.DataFrame(list(zip(city_names, popularity)), columns = ['City', 'Popularity']) 

In [7]:
new_df.head()

,City,Popularity
0,Assen,0
1,Coevorden,0
2,Emmen,0
3,Hoogeveen,0
4,Meppel,0


In [8]:
new_df['City'] = city_names

In [9]:
new_df['Popularity'] = popularity

In [10]:
new_df.head()

,City,Popularity
0,Assen,0
1,Coevorden,0
2,Emmen,0
3,Hoogeveen,0
4,Meppel,0


In [11]:
for index, row in new_df.iterrows():
    # print(row['c1'], row['c2'])
    popularity = pytrend.build_payload([row['City']], cat=0, timeframe='today 5-y', geo='', gprop='')
    interest_df = pytrend.interest_over_time()
    interest = mean(interest_df[row['City']])
    print(interest)
    new_df.loc[index, 'Popularity'] = interest

20.203846153846154
48.12307692307692
60.12307692307692
76.37307692307692
51.026923076923076
73.45384615384616
73.08461538461539
73.70769230769231
65.5576923076923
18.1
17.546153846153846
62.59615384615385
43.57692307692308
46.63846153846154
67.75769230769231
76.48076923076923
77.82307692307693
25.023076923076925
54.52307692307692
61.15384615384615
76
43.10769230769231
80.51923076923077
68.47692307692307
63.83461538461538
69.88461538461539
77.01153846153846
62.90384615384615
48.823076923076925
67.76923076923077
74.17307692307692
49.22692307692308
69.62307692307692
65.31153846153846
68.42692307692307
72.03846153846153
80.19615384615385
50.565384615384616
74.29230769230769
63.2
48.91153846153846
64.26923076923077
66.03076923076924
61.40384615384615
72.69615384615385
46.63076923076923
71.35384615384615
58.62692307692308
61.68846153846154
72.3923076923077
75
73.45769230769231
73.73076923076923
80.09615384615384
47.11153846153846
76.04615384615384
79.71923076923076
44.003846153846155
76.4115

ResponseError: The request failed: Google returned a response with code 429.

In [12]:
new_df.head()

,City,Popularity
0,Assen,20.203846
1,Coevorden,48.123077
2,Emmen,60.123077
3,Hoogeveen,76.373077
4,Meppel,51.026923


In [ ]:
all_jobs = len_city_names * len_city_names
count = 1
toolbar_width = 40


for col in city_names:
    column = []
    start_time = time.time()
    for row in city_names:
        
        uri = "https://www.distance24.org/route.json?stops=" + str(col).strip() + '|' + str(row).strip()
        url = uri.strip()
        try:
            uResponse = requests.get(url)
        except requests.ConnectionError:
            print ('error')  
        Jresponse = uResponse.text
        data = json.loads(Jresponse)
        distance = data['distance']
        column.append(distance)
        division = count/all_jobs
        #print (str(count) + '/' + str(all_jobs))
        count+=1
        #print(column)
    end_time = time.time()
    print("total time taken this loop: ", end_time - start_time)
    new_df[col] = column
    print(new_df.head())

total time taken this loop:  67.66542434692383
        City  Popularity  Assen  Coevorden
0      Assen   20.203846      0         39
1  Coevorden   48.123077     39          0
2      Emmen   60.123077     32         17
3  Hoogeveen   76.373077     30         19
4     Meppel   51.026923     41         37
total time taken this loop:  82.83741307258606
        City  Popularity  Assen  Coevorden
0      Assen   20.203846      0         39
1  Coevorden   48.123077     39          0
2      Emmen   60.123077     32         17
3  Hoogeveen   76.373077     30         19
4     Meppel   51.026923     41         37
total time taken this loop:  84.74003291130066
        City  Popularity  Assen  Coevorden  Emmen
0      Assen   20.203846      0         39     32
1  Coevorden   48.123077     39          0     17
2      Emmen   60.123077     32         17      0
3  Hoogeveen   76.373077     30         19     29
4     Meppel   51.026923     41         37     48
total time taken this loop:  83.87335634231

4          62  
total time taken this loop:  72.35260558128357
        City  Popularity  Assen  Coevorden  Emmen  Hoogeveen  Meppel  Almere  \
0      Assen   20.203846      0         39     32         30      41     114   
1  Coevorden   48.123077     39          0     17         19      37     108   
2      Emmen   60.123077     32         17      0         29      48     122   
3  Hoogeveen   76.373077     30         19     29          0      19      94   
4     Meppel   51.026923     41         37     48         19       0      75   

   Dronten  Emmeloord  Lelystad  Urk  Zeewolde  Drachten  Heerenveen  \
0       76         63        91   74       104        34          43   
1       70         67        87   77        93        66          64   
2       84         77       101   88       108        64          68   
3       55         49        72   59        80        50          46   
4       37         30        53   40        63        46          35   

   Leeuwarden  Sneek  


[5 rows x 23 columns]
total time taken this loop:  82.20302629470825
        City  Popularity  Assen  Coevorden  Emmen  Hoogeveen  Meppel  Almere  \
0      Assen   20.203846      0         39     32         30      41     114   
1  Coevorden   48.123077     39          0     17         19      37     108   
2      Emmen   60.123077     32         17      0         29      48     122   
3  Hoogeveen   76.373077     30         19     29          0      19      94   
4     Meppel   51.026923     41         37     48         19       0      75   

   Dronten  Emmeloord  ...  Heerenveen  Leeuwarden  Sneek  Apeldoorn  Arnhem  \
0       76         63  ...          43          56     61         96     121   
1       70         67  ...          64          87     84         72      95   
2       84         77  ...          68          87     88         90     112   
3       55         49  ...          46          70     65         67      91   
4       37         30  ...          35          62

[5 rows x 30 columns]
total time taken this loop:  87.35928130149841
        City  Popularity  Assen  Coevorden  Emmen  Hoogeveen  Meppel  Almere  \
0      Assen   20.203846      0         39     32         30      41     114   
1  Coevorden   48.123077     39          0     17         19      37     108   
2      Emmen   60.123077     32         17      0         29      48     122   
3  Hoogeveen   76.373077     30         19     29          0      19      94   
4     Meppel   51.026923     41         37     48         19       0      75   

   Dronten  Emmeloord  ...  Culemborg  Doetinchem  Duiven   Ede  Elst  Ermelo  \
0       76         63  ...        146         116     122  6474   129     100   
1       70         67  ...        129          83      93  6497   103      86   
2       84         77  ...        146         100     110  6503   120     102   
3       55         49  ...        120          86      92  6478   100      75   
4       37         30  ...        105        

[5 rows x 37 columns]
total time taken this loop:  98.43658256530762
        City  Popularity  Assen  Coevorden  Emmen  Hoogeveen  Meppel  Almere  \
0      Assen   20.203846      0         39     32         30      41     114   
1  Coevorden   48.123077     39          0     17         19      37     108   
2      Emmen   60.123077     32         17      0         29      48     122   
3  Hoogeveen   76.373077     30         19     29          0      19      94   
4     Meppel   51.026923     41         37     48         19       0      75   

   Dronten  Emmeloord  ...  Nijkerk  Nijmegen  Nunspeet  Putten  Tiel  \
0       76         63  ...      113       137        87     104   145   
1       70         67  ...       98       109        73      89   124   
2       84         77  ...      115       127        89     105   141   
3       55         49  ...       88       107        62      79   118   
4       37         30  ...       72        98        46      63   104   

   Wagening

[5 rows x 43 columns]
total time taken this loop:  91.18866181373596
        City  Popularity  Assen  Coevorden  Emmen  Hoogeveen  Meppel  Almere  \
0      Assen   20.203846      0         39     32         30      41     114   
1  Coevorden   48.123077     39          0     17         19      37     108   
2      Emmen   60.123077     32         17      0         29      48     122   
3  Hoogeveen   76.373077     30         19     29          0      19      94   
4     Meppel   51.026923     41         37     48         19       0      75   

   Dronten  Emmeloord  ...  Wijchen  Winterswijk  Zaltbommel  Zevenaar  \
0       76         63  ...      144          114         159       123   
1       70         67  ...      117           77         139        93   
2       84         77  ...      135           91         156       110   
3       55         49  ...      114           85         132        93   
4       37         30  ...      104           88         118        86   

   Zu

total time taken this loop:  93.64871096611023
        City  Popularity  Assen  Coevorden  Emmen  Hoogeveen  Meppel  Almere  \
0      Assen   20.203846      0         39     32         30      41     114   
1  Coevorden   48.123077     39          0     17         19      37     108   
2      Emmen   60.123077     32         17      0         29      48     122   
3  Hoogeveen   76.373077     30         19     29          0      19      94   
4     Meppel   51.026923     41         37     48         19       0      75   

   Dronten  Emmeloord  ...  Groningen  Hoogezand  Veendam  Brunssum  Geleen  \
0       76         63  ...         25         23       24       232     231   
1       70         67  ...         63         56       50       199     199   
2       84         77  ...         53         43       36       215     215   
3       55         49  ...         55         52       50       202     201   
4       37         30  ...         63         64       64       196     194  